In [170]:
import pandas as pd
import numpy as np
from scipy import stats


In [171]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [203]:
df = pd.read_csv('/content/drive/MyDrive/layer-9/train.csv')
valid = pd.read_csv('/content/drive/MyDrive/layer-9/valid.csv')
test = pd.read_csv('/content/drive/MyDrive/layer-9/test.csv')


In [173]:
df

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,0.019301,0.059756,0.081375,0.057481,-0.068440,-0.165913,0.035643,-0.091138,0.021688,0.057158,...,-0.035576,0.127319,0.098128,-0.058787,0.100971,-0.047754,45,NaN,1,6
1,0.049741,0.090030,0.035118,-0.013676,-0.194317,-0.101763,0.085875,-0.081317,0.112418,0.120523,...,0.020538,0.058968,0.029803,0.111324,0.036727,0.031927,45,NaN,1,6
2,0.019212,0.087779,0.093907,-0.033738,-0.141409,-0.062881,-0.071402,-0.006599,0.020372,-0.027777,...,0.119645,-0.040861,0.000548,-0.061003,-0.042450,0.063340,45,NaN,1,6
3,0.070283,0.049040,0.042126,0.122637,-0.056964,-0.113700,0.108454,0.051336,0.086610,0.141578,...,-0.124494,-0.169225,-0.046391,0.148787,0.014616,-0.140644,45,NaN,1,6
4,0.028864,0.165634,0.016302,0.036117,-0.028871,-0.147748,0.053180,0.025071,-0.004200,-0.022183,...,-0.124862,0.044907,0.084005,-0.038450,0.084371,-0.072146,45,NaN,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28515,-0.041520,0.302638,0.004811,0.211441,-0.249326,-0.113395,-0.146776,-0.146222,-0.040344,-0.124016,...,0.290069,-0.223051,-0.053575,-0.230222,-0.451943,0.117817,39,29.0,1,6
28516,-0.056359,0.061850,0.051865,0.032107,-0.005930,-0.040313,0.117453,-0.067553,0.045004,0.035735,...,0.004518,0.035248,-0.047345,-0.006539,-0.025633,0.002474,39,29.0,1,6
28517,0.019361,0.148111,0.093434,0.011795,-0.074029,-0.077628,0.049933,-0.096354,0.138556,0.130414,...,0.014918,0.015676,-0.033608,0.068212,0.049871,-0.027607,39,29.0,1,6
28518,-0.043644,0.084073,0.074749,0.091776,-0.090782,-0.163187,-0.012568,-0.051021,0.038726,0.011009,...,-0.098420,0.113067,0.028218,0.036682,0.030056,-0.084346,39,29.0,1,6


## **Pre-processing**

In [174]:
# Count missing values in the selected label column
missing_values_label_1 = df["label_1"].isnull().sum()
missing_values_label_2 = df["label_2"].isnull().sum()
missing_values_label_3 = df["label_3"].isnull().sum()
missing_values_label_4 = df["label_4"].isnull().sum()

print(missing_values_label_1)
print(missing_values_label_2)
print(missing_values_label_3)
print(missing_values_label_4)


0
480
0
0


In [200]:
#filling missing values with mode
df.label_2.value_counts()

26.0    4762
25.0    2849
27.0    2846
23.0    2842
31.0    2385
24.0    1906
28.0    1899
30.0    1894
22.0    1432
29.0    1424
33.0     945
36.0     481
35.0     480
34.0     478
32.0     476
41.0     474
61.0     467
Name: label_2, dtype: int64

In [201]:
# Calculate the mean of the column
mean_value = df.label_2.mean()

# Fill missing values in the specified column with the rounded mean
# df.label_2.fillna(round(mean_value), inplace=True)


In [177]:
df_copy =df.copy()
X=df_copy.drop(columns=['label_1','label_2','label_3','label_4'])

#remove outliers
# df=df[(np.abs(stats.zscore(X)) < 3).all(axis=1)]


## **Feature Engineering**

In [178]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


## Label-1 calculation

In [179]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

df_1=df.copy()
X_1=df_1.drop(columns=['label_1','label_2','label_3','label_4'])
y_1=df_1.label_1

knn_1=KNeighborsClassifier()
cross_val_score(knn_1, X_1, y_1, cv=5, scoring='f1_macro').mean()


0.8864463545669438

In [180]:
discrete_features = X_1.dtypes == int

mi_scores = make_mi_scores(X_1, y_1, discrete_features)
mi_scores.sort_values(ascending=False)

feature_242    0.297155
feature_582    0.246642
feature_545    0.208249
feature_204    0.194753
feature_697    0.193833
                 ...   
feature_422    0.032061
feature_65     0.031833
feature_634    0.031481
feature_596    0.031265
feature_138    0.031092
Name: MI Scores, Length: 768, dtype: float64

In [181]:
# Get the last 100 feature names
last_100_features = mi_scores.index[-100:]

# Print the last 100 feature names
print(last_100_features)

Index(['feature_254', 'feature_637', 'feature_112', 'feature_197',
       'feature_602', 'feature_641', 'feature_157', 'feature_366',
       'feature_116', 'feature_45', 'feature_665', 'feature_308',
       'feature_459', 'feature_651', 'feature_277', 'feature_639',
       'feature_361', 'feature_226', 'feature_568', 'feature_318',
       'feature_101', 'feature_80', 'feature_305', 'feature_513',
       'feature_114', 'feature_1', 'feature_237', 'feature_670', 'feature_183',
       'feature_471', 'feature_363', 'feature_231', 'feature_495',
       'feature_93', 'feature_522', 'feature_612', 'feature_699',
       'feature_313', 'feature_257', 'feature_150', 'feature_338',
       'feature_236', 'feature_4', 'feature_299', 'feature_364', 'feature_477',
       'feature_722', 'feature_693', 'feature_674', 'feature_720',
       'feature_509', 'feature_624', 'feature_295', 'feature_282',
       'feature_255', 'feature_233', 'feature_2', 'feature_78', 'feature_726',
       'feature_342', 'feat

In [182]:
X_1 = X_1.drop(columns=last_100_features)


In [183]:
cross_val_score(knn_1, X_1, y_1, cv=5, scoring='f1_macro').mean()

0.888893763424484

In [184]:
knn_1.fit(X_1, y_1)

KNeighborsClassifier()

In [185]:
X_test_1 = test.copy().drop(columns=['ID'])
X_test_1 = X_test_1.drop(columns=last_100_features)

In [186]:
label_1_pred = knn_1.predict(X_test_1)

In [187]:
label_1_pred

array([26, 18, 16,  7, 58, 46,  7, 22, 29, 26, 33,  6, 54, 51, 29, 48, 23,
        2, 52, 42, 11, 35, 39,  2, 44, 33, 28, 55,  2, 51, 55, 46,  1, 56,
       30, 23, 50,  6, 40, 18, 48, 47,  6, 46, 59, 25, 31, 35,  5, 32, 25,
       56, 39, 37, 32, 29, 46, 52, 28, 34, 16, 47, 55, 24, 23, 18, 20,  6,
       36, 38, 22, 28, 44, 46, 55, 59, 49, 40, 57, 12, 52, 38, 29,  4, 31,
       49, 14, 19, 12, 39, 36, 43, 34, 32, 54, 46, 27,  5,  7, 42, 19, 26,
       31, 36, 53, 10, 12, 33, 38,  2, 57, 37, 17, 50, 46, 46, 17, 34,  2,
       40, 48, 40,  8, 22, 38, 15, 39,  9, 52,  5, 40, 13, 10, 55, 28, 25,
       42, 57, 16, 27, 23,  2, 51, 27, 32, 44, 16,  4, 11, 46, 42, 37, 47,
       50,  3, 41, 19, 20, 15,  8,  3, 18,  5, 28, 46, 49, 16, 40, 17, 20,
       20, 57, 27, 16, 11, 59, 34, 52, 41, 55,  7, 18, 21,  4, 37, 55, 29,
       28, 23, 35, 46, 52, 30, 27, 40,  1, 28, 14,  3, 58, 17, 60, 18,  3,
       22,  9,  5, 25, 17, 26, 19, 36, 51, 21,  1, 44, 39, 47, 38,  6, 50,
       39, 23, 37, 48, 22

## Label-4 calculation

In [188]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.utils import resample


df_4=df.copy()
X_4=df_4.drop(columns=['label_1','label_2','label_3','label_4'])
y_4=df_4.label_4


# Separate the majority class (class 6) from other classes
majority_class = 6
minority_classes = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13]

# Create DataFrames for majority and minority classes
df_majority = df_4[df_4['label_4'] == majority_class]
df_minority = df_4[df_4['label_4'].isin(minority_classes)]

# Down-sample the majority class to match the size of minority classes
num_samples_minority = len(df_minority)
df_majority_downsampled = resample(df_majority, replace=False, n_samples=num_samples_minority, random_state=42)

# Combine minority and down-sampled majority classes
df_balanced = pd.concat([df_minority, df_majority_downsampled])

# Shuffle the balanced dataset
df_4 = df_balanced.sample(frac=1, random_state=42)

# Now df_balanced contains a balanced dataset
df_4.label_4.value_counts()

knn_4=KNeighborsClassifier()
cross_val_score(knn_4, X_4, y_4, cv=5, scoring='f1_macro').mean()


0.8127539825952803

In [189]:
discrete_features = X_4.dtypes == int

mi_scores = make_mi_scores(X_4, y_4, discrete_features)
mi_scores.sort_values(ascending=False)

feature_242    0.070313
feature_582    0.064326
feature_545    0.053757
feature_302    0.052404
feature_633    0.047990
                 ...   
feature_459    0.003709
feature_181    0.003608
feature_634    0.003268
feature_282    0.003099
feature_209    0.001615
Name: MI Scores, Length: 768, dtype: float64

In [190]:
# Get the last 100 feature names
last_100_features = mi_scores.index[-100:]

# Print the last 100 feature names
print(last_100_features)

Index(['feature_590', 'feature_80', 'feature_635', 'feature_29', 'feature_693',
       'feature_109', 'feature_708', 'feature_495', 'feature_348',
       'feature_666', 'feature_474', 'feature_502', 'feature_260',
       'feature_428', 'feature_714', 'feature_674', 'feature_203',
       'feature_101', 'feature_228', 'feature_280', 'feature_231',
       'feature_682', 'feature_300', 'feature_307', 'feature_757',
       'feature_485', 'feature_4', 'feature_398', 'feature_473', 'feature_491',
       'feature_637', 'feature_656', 'feature_741', 'feature_48',
       'feature_520', 'feature_624', 'feature_392', 'feature_201',
       'feature_219', 'feature_680', 'feature_698', 'feature_602',
       'feature_421', 'feature_346', 'feature_317', 'feature_174',
       'feature_66', 'feature_726', 'feature_238', 'feature_55', 'feature_285',
       'feature_374', 'feature_226', 'feature_255', 'feature_372',
       'feature_262', 'feature_388', 'feature_1', 'feature_341', 'feature_88',
       'feat

In [191]:
X_4 = X_4.drop(columns=last_100_features)


In [192]:
cross_val_score(knn_4, X_4, y_4, cv=5, scoring='f1_macro').mean()

0.8149110955007934

In [193]:
knn_4.fit(X_4, y_4)

KNeighborsClassifier()

In [194]:
X_test_4 = test.copy().drop(columns=['ID'])
X_test_4 = X_test_4.drop(columns=last_100_features)

In [195]:
label_4_pred = knn_4.predict(X_test_4)

In [196]:
label_4_pred

array([ 2,  8,  6,  6,  6,  6,  6,  6,  6,  2,  6,  6,  6,  6,  6,  6,  6,
        6,  5,  0,  6,  2,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  8,  6,  3,  6,  6,  6,  1,  6,  2,  6,  0,  1,
        6,  6,  7,  0,  6,  6,  5,  6,  6,  6,  3,  6,  2,  6,  6,  6,  6,
        6, 12,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  5, 12,  6,  6,  6,
        6, 12,  4,  6,  6,  6,  6,  6,  0,  6,  6,  7,  6,  6,  0,  4,  2,
        6,  6,  6,  6,  6,  6, 12,  6,  6,  7,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6, 12,  9,  6, 11,  5,  6,  6,  6,  6,  6,  6,  1,
        0,  6,  6,  7,  6,  6,  6,  7,  0,  6,  6,  6,  6,  6,  0,  7,  3,
        6,  6, 10,  4,  6,  9,  6,  6,  8,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  7,  6,  6,  6,  6,  5, 10,  6,  6,  8,  6,  6,  7,  6,  6,
        6,  6,  2,  6,  5,  6,  7,  6,  6,  6, 12,  6,  6,  6, 13,  8,  6,
        6, 11,  6,  1,  6,  2,  4,  6,  6,  6,  6,  6,  6,  3, 12,  6,  6,
        6,  6,  7,  6,  6

## Label-2 calculation

In [209]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

df_2=df.copy()
df_2=df_2.dropna()
X_2=df_2.drop(columns=['label_1','label_2','label_3','label_4'])
y_2=df_2.label_2


knn_2=KNeighborsClassifier()
cross_val_score(knn_2, X_2, y_2, cv=5, scoring='f1_macro').mean()


0.6837698237178371

In [211]:
discrete_features = X_2.dtypes == int

mi_scores = make_mi_scores(X_2, y_2, discrete_features)
mi_scores.sort_values(ascending=False)

feature_242    0.110355
feature_582    0.105245
feature_545    0.070757
feature_204    0.069773
feature_332    0.058931
                 ...   
feature_567    0.003784
feature_477    0.003592
feature_647    0.003077
feature_596    0.002688
feature_263    0.002428
Name: MI Scores, Length: 768, dtype: float64

In [212]:
# Get the last 100 feature names
last_100_features = mi_scores.index[-100:]

# Print the last 100 feature names
print(last_100_features)

Index(['feature_6', 'feature_123', 'feature_637', 'feature_80', 'feature_134',
       'feature_709', 'feature_201', 'feature_78', 'feature_744',
       'feature_683', 'feature_290', 'feature_595', 'feature_357',
       'feature_503', 'feature_300', 'feature_54', 'feature_243',
       'feature_372', 'feature_354', 'feature_252', 'feature_427',
       'feature_74', 'feature_432', 'feature_75', 'feature_206', 'feature_101',
       'feature_295', 'feature_191', 'feature_20', 'feature_653', 'feature_70',
       'feature_757', 'feature_672', 'feature_236', 'feature_409',
       'feature_114', 'feature_92', 'feature_73', 'feature_684', 'feature_231',
       'feature_463', 'feature_622', 'feature_210', 'feature_151',
       'feature_282', 'feature_116', 'feature_524', 'feature_255',
       'feature_149', 'feature_227', 'feature_768', 'feature_22',
       'feature_174', 'feature_205', 'feature_491', 'feature_606',
       'feature_200', 'feature_753', 'feature_60', 'feature_93', 'feature_44',
  

In [214]:
X_2 = X_2.drop(columns=last_100_features)


In [215]:
cross_val_score(knn_2, X_2, y_2, cv=5, scoring='f1_macro').mean()

0.6844346634599926

In [218]:
knn_2.fit(X_2, y_2)

KNeighborsClassifier()

In [219]:
X_test_2 = test.copy().drop(columns=['ID'])
X_test_2 = X_test_2.drop(columns=last_100_features)

In [220]:
label_2_pred = knn_2.predict(X_test_2)

In [222]:
label_2_pred

array([22., 25., 30., 27., 29., 30., 27., 33., 23., 22., 26., 25., 27.,
       26., 23., 26., 28., 25., 34., 29., 33., 24., 29., 25., 61., 26.,
       28., 23., 25., 26., 23., 30., 30., 24., 28., 28., 24., 25., 26.,
       25., 26., 23., 25., 30., 31., 22., 26., 24., 25., 23., 22., 24.,
       29., 27., 23., 23., 30., 34., 28., 25., 30., 23., 23., 26., 28.,
       25., 25., 25., 22., 32., 33., 28., 61., 30., 23., 26., 26., 26.,
       27., 26., 34., 32., 23., 23., 26., 26., 31., 23., 26., 29., 22.,
       31., 25., 23., 27., 30., 31., 25., 27., 29., 23., 22., 26., 22.,
       24., 36., 26., 26., 32., 25., 27., 27., 26., 24., 30., 30., 26.,
       25., 25., 26., 29., 26., 41., 33., 32., 28., 29., 35., 34., 25.,
       26., 27., 36., 23., 28., 22., 23., 27., 30., 31., 28., 25., 26.,
       31., 23., 30., 30., 23., 33., 30., 29., 27., 23., 24., 31., 30.,
       23., 25., 28., 41., 31., 25., 25., 28., 30., 26., 30., 26., 26.,
       25., 25., 27., 31., 30., 33., 31., 25., 34., 30., 23., 27

## Label-3 calculation

In [223]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

df_3=df.copy()
X_3=df_3.drop(columns=['label_1','label_2','label_3','label_4'])
y_3=df_3.label_3


knn_3=KNeighborsClassifier()
cross_val_score(knn_3, X_3, y_3, cv=5, scoring='f1_macro').mean()


0.9031988529340838

In [224]:
discrete_features = X_3.dtypes == int

mi_scores = make_mi_scores(X_3, y_3, discrete_features)
mi_scores.sort_values(ascending=False)

feature_630    0.073795
feature_242    0.067246
feature_582    0.062053
feature_426    0.061418
feature_671    0.048750
                 ...   
feature_14     0.000000
feature_88     0.000000
feature_78     0.000000
feature_77     0.000000
feature_768    0.000000
Name: MI Scores, Length: 768, dtype: float64

In [225]:
# Get the last 100 feature names
last_100_features = mi_scores.index[-100:]

# Print the last 100 feature names
print(last_100_features)

Index(['feature_444', 'feature_296', 'feature_365', 'feature_166',
       'feature_130', 'feature_550', 'feature_385', 'feature_99',
       'feature_366', 'feature_5', 'feature_604', 'feature_101', 'feature_507',
       'feature_650', 'feature_201', 'feature_600', 'feature_665',
       'feature_308', 'feature_440', 'feature_537', 'feature_181',
       'feature_152', 'feature_87', 'feature_180', 'feature_744',
       'feature_475', 'feature_755', 'feature_639', 'feature_217',
       'feature_733', 'feature_224', 'feature_213', 'feature_211', 'feature_4',
       'feature_192', 'feature_227', 'feature_167', 'feature_237',
       'feature_149', 'feature_25', 'feature_142', 'feature_120', 'feature_53',
       'feature_760', 'feature_759', 'feature_65', 'feature_66', 'feature_77',
       'feature_78', 'feature_88', 'feature_14', 'feature_93', 'feature_255',
       'feature_749', 'feature_12', 'feature_110', 'feature_20', 'feature_254',
       'feature_298', 'feature_260', 'feature_483', 'fea

In [228]:
X_3 = X_3.drop(columns=last_100_features)


In [229]:
cross_val_score(knn_3, X_3, y_3, cv=5, scoring='f1_macro').mean()

0.907970035720945

In [230]:
knn_3.fit(X_3, y_3)

KNeighborsClassifier()

In [231]:
X_test_3 = test.copy().drop(columns=['ID'])
X_test_3 = X_test_3.drop(columns=last_100_features)

In [232]:
label_3_pred = knn_3.predict(X_test_3)

In [233]:
label_3_pred

array([0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## **final output**

In [235]:
new_df = pd.DataFrame({"ID": test["ID"], 'label_1': label_1_pred ,'label_2': label_2_pred ,'label_3': label_3_pred ,'label_4': label_4_pred })


In [236]:
new_df

,ID,label_1,label_2,label_3,label_4
0,1,26,22.0,0,2
1,2,18,25.0,1,8
2,3,16,30.0,1,6
3,4,7,27.0,1,6
4,5,58,29.0,0,6
...,...,...,...,...,...
739,740,5,26.0,1,6
740,741,35,24.0,1,2
741,742,54,25.0,1,6
742,743,38,32.0,1,12


In [237]:
new_df.to_csv('ML_project_layer_9.csv',index=False)